In [1]:
import os
# os.chdir(r'C:/Users/hakan/Jupyter Notebook/notebook/AIFarming/DataAnalysis/')
# os.getcwd()


In [2]:
import pandas as pd
import numpy as np

data_path ='C:/Users/hakan/Jupyter Notebook/data/porto-seguro-safe-driver-prediction/'
train = pd.read_csv(data_path+'train.csv',index_col='id')
test = pd.read_csv(data_path+'test.csv',index_col='id')
submission = pd.read_csv(data_path+'sample_submission.csv',index_col='id')

In [3]:
def resumatable(df):
    print(f'DataFrame Shape: {df.shape}')
    summary = pd.DataFrame(df.dtypes, columns=['Dtype'])
    summary['Missing'] = (df == -1).sum().values
    summary['Unique'] = df.nunique().values
    summary['Dtype'] = None
    for col in df.columns:
        if 'bin' in col or col =='target':
            summary.loc[col,'Dtype'] = 'Binary'
        elif 'cat' in col:
            summary.loc[col,'Dtype'] = 'Categorical'
        elif df[col].dtype == 'float64':
            summary.loc[col,'Dtype'] = 'Continuous'
        elif df[col].dtype == 'int64':
            summary.loc[col,'Dtype'] = 'Ordinal'
    return summary

In [4]:
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

#feature engineering

In [5]:
import pandas as pd
data_path ='C:/Users/hakan/Jupyter Notebook/data/porto-seguro-safe-driver-prediction/'
train = pd.read_csv(data_path+'train.csv',index_col='id')
test = pd.read_csv(data_path+'test.csv',index_col='id')
submission = pd.read_csv(data_path+'sample_submission.csv',index_col='id')

In [6]:
all_data = pd.concat([train,test],ignore_index=True)
all_data = all_data.drop('target',axis=1)

In [7]:
all_features = all_data.columns
all_features

Index(['ps_ind_01', 'ps_ind_02_cat', 'ps_ind_03', 'ps_ind_04_cat',
       'ps_ind_05_cat', 'ps_ind_06_bin', 'ps_ind_07_bin', 'ps_ind_08_bin',
       'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_12_bin',
       'ps_ind_13_bin', 'ps_ind_14', 'ps_ind_15', 'ps_ind_16_bin',
       'ps_ind_17_bin', 'ps_ind_18_bin', 'ps_reg_01', 'ps_reg_02', 'ps_reg_03',
       'ps_car_01_cat', 'ps_car_02_cat', 'ps_car_03_cat', 'ps_car_04_cat',
       'ps_car_05_cat', 'ps_car_06_cat', 'ps_car_07_cat', 'ps_car_08_cat',
       'ps_car_09_cat', 'ps_car_10_cat', 'ps_car_11_cat', 'ps_car_11',
       'ps_car_12', 'ps_car_13', 'ps_car_14', 'ps_car_15', 'ps_calc_01',
       'ps_calc_02', 'ps_calc_03', 'ps_calc_04', 'ps_calc_05', 'ps_calc_06',
       'ps_calc_07', 'ps_calc_08', 'ps_calc_09', 'ps_calc_10', 'ps_calc_11',
       'ps_calc_12', 'ps_calc_13', 'ps_calc_14', 'ps_calc_15_bin',
       'ps_calc_16_bin', 'ps_calc_17_bin', 'ps_calc_18_bin', 'ps_calc_19_bin',
       'ps_calc_20_bin'],
      dtype='obj

In [8]:
from sklearn.preprocessing import OneHotEncoder

#one hot encoding for categorical datas
cat_features = [feature for feature in all_features if 'cat' in feature]
onehot_encoder = OneHotEncoder()

encoded_cat_matrix = onehot_encoder.fit_transform(all_data[cat_features])

In [9]:
drop_features = ['ps_ind_10_bin','ps_ind_11_bin','ps_ind_12_bin','ps_ind_13_bin','ps_ind_14','ps_car_14']

In [10]:
remaining_features = [feature for feature in all_features if ('cat' not in feature and 'calc' not in feature and feature not in drop_features)]

In [11]:
remaining_features

['ps_ind_01',
 'ps_ind_03',
 'ps_ind_06_bin',
 'ps_ind_07_bin',
 'ps_ind_08_bin',
 'ps_ind_09_bin',
 'ps_ind_15',
 'ps_ind_16_bin',
 'ps_ind_17_bin',
 'ps_ind_18_bin',
 'ps_reg_01',
 'ps_reg_02',
 'ps_reg_03',
 'ps_car_11',
 'ps_car_12',
 'ps_car_13',
 'ps_car_15']

Sparse한 (ex : Onehot encoded) matrix를 다룰 때 scipy.sparse를 사용한다. 대규모 행렬을 다룰 때 메모리 문제를 해결하기 위해 사용한다. scipy.sparse의 matrix를 만드는 방법은 sparse matrix 구성요소를 직접 입력하는 방법과 numpy.ndarray를 입력하는 방법이 있다.

In [12]:
from scipy import  sparse
all_data_sprs = sparse.hstack([sparse.csr_matrix(all_data[remaining_features]),encoded_cat_matrix],format='csr')

In [13]:
all_data_sprs.shape, train.shape, test.shape

((1488028, 201), (595212, 58), (892816, 57))

In [14]:
num_train = len(train)
print(num_train)

X = all_data_sprs[:num_train]
X_test = all_data_sprs[num_train:]

y = train['target'].values

595212


In [15]:
import numpy as np
def eval_gini(y_true, y_pred):
    assert y_true.shape == y_pred.shape

    n_samples = y_true.shape[0]
    L_mid = np.linspace(1/n_samples, 1, n_samples)

    pred_order = y_true[y_pred.argsort()]
    true_order = y_true[y_true.argsort()]
    
    L_pred = np.cumsum(pred_order)/np.sum(true_order)
    G_pred = np.sum(L_mid - L_pred)
    
    
    L_true = np.cumsum(true_order)/np.sum(true_order)
    G_true = np.sum(L_mid - L_true)

    #Normalized Gini coefficient
    return G_pred/G_true    

In [16]:
#gini for LGBM
def gini(preds,dtrain):
    labels = dtrain.get_label()
    return 'gini', eval_gini(labels,preds), True

In [17]:
from sklearn.model_selection import StratifiedKFold

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=1991)

In [18]:
print(folds.split(X,y))

<generator object _BaseKFold.split at 0x000001C22A04A350>


In [19]:
lgb_params = {
    'objective':'binary',
    'learning_rate':0.01,
    'force_row_wise':True,
    'random_state':0,
    'scale_pos_weight': 2,
}

In [20]:
oof_val_preds = np.zeros(X.shape[0])
oof_test_preds = np.zeros(X_test.shape[0])

In [21]:
import lightgbm as lgb
#Normal Test
for idx, (train_idx,valid_idx) in enumerate(folds.split(X,y)):
    print('#'*40,f'폴드 {idx+1} / {folds.n_splits}','#'*40)
    #Train, Validation data
    X_train, y_train = X[train_idx],y[train_idx]
    X_valid, y_valid = X[valid_idx],y[valid_idx]

    #data set for lgbm
    lgb_train = lgb.Dataset(X_train,y_train)
    lgb_valid = lgb.Dataset(X_valid,y_valid)

    # train lgbm model
    lgb_model = lgb.train(params=lgb_params,
                         train_set = lgb_train,
                         num_boost_round=1000,
                         valid_sets=lgb_valid,
                         feval = gini,                                                   
                         callbacks=[lgb.early_stopping(stopping_rounds=5)])

    #oof prediction based on test data
    oof_test_preds += lgb_model.predict(X_test)/folds.n_splits
    #based in validation data
    oof_val_preds[valid_idx] += lgb_model.predict(X_valid)

    #gini
    gini_score = eval_gini(y_valid, oof_val_preds[valid_idx])
    print(f'fold {idx+1} gini score : {gini_score}\n')

######################################## 폴드 1 / 5 ########################################
[LightGBM] [Info] Number of positive: 17355, number of negative: 458814
[LightGBM] [Info] Total Bins 1095
[LightGBM] [Info] Number of data points in the train set: 476169, number of used features: 200
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.036447 -> initscore=-3.274764
[LightGBM] [Info] Start training from score -3.274764
Training until validation scores don't improve for 5 rounds
Early stopping, best iteration is:
[24]	valid_0's binary_logloss: 0.155301	valid_0's gini: 0.243445
fold 1 gini score : 0.2434451362266238

######################################## 폴드 2 / 5 ########################################
[LightGBM] [Info] Number of positive: 17355, number of negative: 458814
[LightGBM] [Info] Total Bins 1093
[LightGBM] [Info] Number of data points in the train set: 476169, number of used features: 200
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.036447 -> initscore=-3.274764
[L

In [22]:
#is_unbalance
for idx, (train_idx,valid_idx) in enumerate(folds.split(X,y)):
    print('#'*40,f'폴드 {idx+1} / {folds.n_splits}','#'*40)
    #Train, Validation data
    X_train, y_train = X[train_idx],y[train_idx]
    X_valid, y_valid = X[valid_idx],y[valid_idx]

    #data set for lgbm
    lgb_train = lgb.Dataset(X_train,y_train)
    lgb_valid = lgb.Dataset(X_valid,y_valid)

    # train lgbm model
    lgb_model = lgb.train(params=lgb_params,
                         train_set = lgb_train,
                         num_boost_round=1000,
                         valid_sets=lgb_valid,
                         feval = gini,                            
                         callbacks=[lgb.early_stopping(stopping_rounds=5)])

    #oof prediction based on test data
    oof_test_preds += lgb_model.predict(X_test)/folds.n_splits
    #based in validation data
    oof_val_preds[valid_idx] += lgb_model.predict(X_valid)

    #gini
    gini_score = eval_gini(y_valid, oof_val_preds[valid_idx])
    print(f'fold {idx+1} gini score : {gini_score}\n')

######################################## 폴드 1 / 5 ########################################
[LightGBM] [Info] Number of positive: 17355, number of negative: 458814
[LightGBM] [Info] Total Bins 1095
[LightGBM] [Info] Number of data points in the train set: 476169, number of used features: 200
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.036447 -> initscore=-3.274764
[LightGBM] [Info] Start training from score -3.274764
Training until validation scores don't improve for 5 rounds
Early stopping, best iteration is:
[24]	valid_0's binary_logloss: 0.155301	valid_0's gini: 0.243445
fold 1 gini score : 0.2434451362266238

######################################## 폴드 2 / 5 ########################################
[LightGBM] [Info] Number of positive: 17355, number of negative: 458814
[LightGBM] [Info] Total Bins 1093
[LightGBM] [Info] Number of data points in the train set: 476169, number of used features: 200
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.036447 -> initscore=-3.274764
[L

In [23]:
lgb_params = {
    'objective':'binary',
    'learning_rate':0.01,
    'force_row_wise':True,
    'random_state':0,
    'scale_pos_weight': 10,
}

#scale_pos_weight :1.5~10 
for idx, (train_idx,valid_idx) in enumerate(folds.split(X,y)):
    print('#'*40,f'폴드 {idx+1} / {folds.n_splits}','#'*40)
    #Train, Validation data
    X_train, y_train = X[train_idx],y[train_idx]
    X_valid, y_valid = X[valid_idx],y[valid_idx]

    #data set for lgbm
    lgb_train = lgb.Dataset(X_train,y_train)
    lgb_valid = lgb.Dataset(X_valid,y_valid)

    # train lgbm model
    lgb_model = lgb.train(params=lgb_params,
                         train_set = lgb_train,
                         num_boost_round=1000,
                         valid_sets=lgb_valid,
                         feval = gini,                                                   
                         callbacks=[lgb.early_stopping(stopping_rounds=5)])

    #oof prediction based on test data
    oof_test_preds += lgb_model.predict(X_test)/folds.n_splits
    #based in validation data
    oof_val_preds[valid_idx] += lgb_model.predict(X_valid)

    #gini
    gini_score = eval_gini(y_valid, oof_val_preds[valid_idx])
    print(f'fold {idx+1} gini score : {gini_score}\n')

######################################## 폴드 1 / 5 ########################################
[LightGBM] [Info] Number of positive: 17355, number of negative: 458814
[LightGBM] [Info] Total Bins 1095
[LightGBM] [Info] Number of data points in the train set: 476169, number of used features: 200
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.036447 -> initscore=-3.274764
[LightGBM] [Info] Start training from score -3.274764
Training until validation scores don't improve for 5 rounds
Early stopping, best iteration is:
[2]	valid_0's binary_logloss: 0.156172	valid_0's gini: 0.22999
fold 1 gini score : 0.24268400803554507

######################################## 폴드 2 / 5 ########################################
[LightGBM] [Info] Number of positive: 17355, number of negative: 458814
[LightGBM] [Info] Total Bins 1093
[LightGBM] [Info] Number of data points in the train set: 476169, number of used features: 200
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.036447 -> initscore=-3.274764
[Li

In [24]:
print(eval_gini(y,oof_val_preds))

0.24051405638858542


In [25]:
print(np.sum(np.round(oof_test_preds))/len(oof_test_preds))

0.0


In [26]:
submission['target'] = oof_test_preds
submission.to_csv('base_submission.csv')

In [27]:
from catboost import CatBoostClassifier, Pool

# test_data = Pool(X,y)
model = CatBoostClassifier(iterations =1000, depth=5,learning_rate = 0.01,loss_function='Logloss',verbose=100,od_pval=0.01)

history = model.fit(X,y)
preds_class = model.predict(X_test)
preds_proba = model.predict_proba(X_test)
print('class : ',preds_class)
print('proba : ',preds_proba)

0:	learn: 0.6779991	total: 166ms	remaining: 2m 45s
100:	learn: 0.1906039	total: 2.85s	remaining: 25.3s
200:	learn: 0.1573332	total: 5.55s	remaining: 22.1s
300:	learn: 0.1532955	total: 8.3s	remaining: 19.3s
400:	learn: 0.1524549	total: 11.1s	remaining: 16.6s
500:	learn: 0.1521069	total: 13.9s	remaining: 13.8s
600:	learn: 0.1518821	total: 16.7s	remaining: 11.1s
700:	learn: 0.1517073	total: 19.5s	remaining: 8.33s
800:	learn: 0.1515623	total: 22.4s	remaining: 5.55s
900:	learn: 0.1514323	total: 25.2s	remaining: 2.77s
999:	learn: 0.1513168	total: 28.1s	remaining: 0us
class :  [0 0 0 ... 0 0 0]
proba :  [[0.97463692 0.02536308]
 [0.97460532 0.02539468]
 [0.97273679 0.02726321]
 ...
 [0.96242168 0.03757832]
 [0.97436631 0.02563369]
 [0.9689999  0.0310001 ]]


In [28]:
print(preds_proba[:,0])
np.round(preds_proba)
print(np.sum(preds_proba[:,0])/preds_proba.shape[0])
print(np.sum(preds_proba[:,1])/preds_proba.shape[0])

[0.97463692 0.97460532 0.97273679 ... 0.96242168 0.97436631 0.9689999 ]
0.9635820950969949
0.036417904903005484
